In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm_notebook(range(page_start, page_end + 1)):
        url_format = url_set.format(x)
        r = requests.get(url_format)
        soup = BeautifulSoup(r.text, 'html.parser')
        table = soup.find('table', attrs={'class': 'table'})
        table_rows = table.find_all('tr')
        
        for rows in table_rows:

        # Scrape Beer Name    
            try:
                beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
            except:
                beer_name.append(None)
        # Scrape Review Rating  
            try:
                beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
            except:
                beer_rating.append(None)
        # Scrape Brewery Name  
            try:
                brewery_div = rows.find('div')
                brewery_name.append(brewery_div.a.text)
            except:
                brewery_name.append(None)
        # Scrape Brewery Location
            try:
                brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
            except:
                brewery_location.append(None)
        # Scrape Reviewer Name
            try:
                reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
            except:
                reviewer_name.append(None)
        # Scrape content
            try:
                review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
            except:
                review_content.append(None)

    jobz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return jobz



In [11]:
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
ratebeer_scraper(1,10,url)

/Users/lukebetham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


,beer_name,beer_rating,brewery_name,brewery_location,review_content,reviewer_name
0,Greene King Yardbird (Bottle / Keg),2.9,Greene King,"Bury St. Edmunds, Suffolk",Bottle. Golden ale with good white foam. Aroma...,alex_leit
1,Greene King St. Edmund's Golden Beer (Bottle),2.6,Greene King,"Bury St. Edmunds, Suffolk","Bottle. Golden color. Aroma: weak, malty, a bi...",alex_leit
2,Fuller's Winter Brew,3.6,Fuller's Brewery (Asahi),"Chiswick, Greater London","Flasche 0,5l: Dunkles Kupfer, Kastanienbraun, ...",Thomas_W1
3,Fuller's Old Winter Ale (Bottle),3.3,Fuller's Brewery (Asahi),"Chiswick, Greater London","Flasche 0,5l: Dunkles Kupfer, Kastanienbraun, ...",Thomas_W1
4,Batemans Rosey Nosey (Bottle),3.0,Batemans,"Wainfleet, Lincolnshire","Flasche 0,5l: Dunkles Kupfer, Mahagoni, glaskl...",Thomas_W1
...,...,...,...,...,...,...
145,Other Half Crickets - Motueka,3.5,Other Half Brewing,"Brooklyn, New York","Can, courtesy of mansquito. Pours gold. Peach,...",explosivedog
146,Dogfish Head World Wide Stout - Bourbon Barrel...,4.1,Dogfish Head Brewery (Boston Beer Co.),"Milton, Delaware","Black color with a thin beige head. Coffee, ch...",sinfull
147,Separatist Euro Spliff,3.6,Separatist Beer Project,"Easton, Pennsylvania",Appears gold with foamy white head; Aroma is g...,closey22
148,Jester King Kvass,3.1,Jester King Brewery,"Austin, Texas","Bottle. Gusher. Light yellow body, white head,...",northropfrye
